In [32]:
def recuperer_valeurs_dataset_en_local(chemin_echantillon_dataset):
    valeurs = []
    for nom_dossier_images_fruit in listdir(chemin_echantillon_dataset):
        chemin_dossier_images_fruit = chemin_echantillon_dataset + nom_dossier_images_fruit + "/"
        for nom_image_fruit in listdir(chemin_dossier_images_fruit):
            chemin_image_fruit = chemin_dossier_images_fruit + nom_image_fruit
            valeurs.append([chemin_image_fruit, nom_dossier_images_fruit])
    return valeurs

def creer_dataset_initial_en_local(chemin_echantillon_dataset):
    colonnes = ["chemin_image", "label"]
    valeurs = recuperer_valeurs_dataset_en_local(chemin_echantillon_dataset)
    return spark.createDataFrame(valeurs, colonnes)

def pretraiter(image):
    """
    Retourne l'image dont on a appliqué des prétraitements.
    
    
    Paramètre :
        - image : image à prétraiter
    """
    image_pretraitee = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_pretraitee = cv.equalizeHist(image_pretraitee)
    image_pretraitee = cv.GaussianBlur(image_pretraitee, (5,5), 3)
    image_pretraitee = cv.resize(image_pretraitee, dsize=(100,100))
    return image_pretraitee

def extraire_descripteurs(nom_fichier_image):
    """
    Retourne les descripteurs associés aux features d'une image à partir du nom de son fichier
    
    
    Paramètre :
        - nom_fichier_image : nom du fichier contenant l'image dont on veut tirer les descripteurs
    """
    image = cv.imread(nom_fichier_image)
    image_pretraitee = pretraiter(image)
    sift = cv.SIFT_create()
    keypoints, descripteurs = sift.detectAndCompute(image_pretraitee, None)
    
    descripteurs = descripteurs.flatten().tolist()
    
    return descripteurs

In [23]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, ArrayType, FloatType
from pyspark.sql.functions import udf
from os import listdir
import cv2 as cv

# Initialisation du programme
#appName = "[OC/P8] - Déployer un modèle sur le cloud"
#master = "local"
#conf = SparkConf().setAppName(appName).setMaster(master)
#sc = SparkContext(conf=conf).getOrCreate()
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
chemin_echantillon_dataset = "data/echantillon_dataset/"

df_images_fruits = creer_dataset_initial_en_local(chemin_echantillon_dataset)
udf_extraire_descripteurs = udf(extraire_descripteurs, ArrayType(FloatType()))

df_images_fruits = df_images_fruits.withColumn("descripteurs", udf_extraire_descripteurs("chemin_image"))
df_images_fruits.show(truncate=True)

+--------------------+------------------+--------------------+
|        chemin_image|             label|        descripteurs|
+--------------------+------------------+--------------------+
|data/echantillon_...|apple_granny_smith|[1.0, 0.0, 0.0, 0...|
|data/echantillon_...|apple_granny_smith|[42.0, 30.0, 0.0,...|
|data/echantillon_...|           avocado|[54.0, 26.0, 0.0,...|
|data/echantillon_...|           avocado|[98.0, 84.0, 0.0,...|
|data/echantillon_...|            banana|[0.0, 0.0, 0.0, 0...|
|data/echantillon_...|            banana|[150.0, 22.0, 0.0...|
+--------------------+------------------+--------------------+



In [31]:
des = df_images_fruits.collect()[1][2]
len(des)

640

In [30]:
len(des)

1280